In [1]:
from pathlib import Path
import os
import sys

definitions_path = Path(sys.prefix) / "share/eccodes-cosmo-resources/definitions"
os.environ["ECCODES_DEFINITION_PATH"] = str(definitions_path)

import dask
import earthkit.data as ekd
import numpy as np
from obstore.store import LocalStore
import xarray as xr
from virtualizarr.registry import ObjectStoreRegistry
from virtualizarr import open_virtual_dataset

from icon_ch_vzarr import IconChParser
from icon_ch_vzarr.parsers import DEFAULT_PROFILE

In [2]:
FILTERS = {"shortName": ["T", "T_2M","W","CLCL","TOT_PREC"]}

FILENAME = "/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/"
files = sorted(Path(FILENAME).glob("iaf202501012*"))
urls = [f"file://{file}" for file in files]

print(f"Found {len(urls)} files")

Found 4 files


In [3]:
source = ekd.from_source("file", files).sel(FILTERS)
ds = source.to_xarray(**DEFAULT_PROFILE)
ds

<xarray.Dataset> Size: 6GB
Dimensions:               (valid_time: 4, values: 1147980,
                           generalVerticalLayer: 80, generalVertical: 81)
Coordinates:
  * valid_time            (valid_time) datetime64[ns] 32B 2025-01-01T20:00:00...
  * generalVerticalLayer  (generalVerticalLayer) int64 640B 1 2 3 4 ... 78 79 80
  * generalVertical       (generalVertical) int64 648B 1 2 3 4 5 ... 78 79 80 81
Dimensions without coordinates: values
Data variables:
    CLCL                  (valid_time, values) float64 37MB ...
    T                     (valid_time, generalVerticalLayer, values) float64 3GB ...
    TOT_PREC              (valid_time, values) float64 37MB ...
    T_2M                  (valid_time, values) float64 37MB ...
    W                     (valid_time, generalVertical, values) float64 3GB ...
Attributes:
    date:         20250101
    time:         2000
    Conventions:  CF-1.8
    institution:  MeteoSwiss

In [4]:
store = LocalStore("/")
registry = ObjectStoreRegistry({"file://": store})
parser = IconChParser(filter_keys=FILTERS)

manifest_store = parser(urls[0], registry)
ds = xr.open_zarr(manifest_store, consolidated=False, decode_times=False, zarr_format=3)
ds

data=MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010120),[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1284, 1437, 1473]),[0, 1, 2, 3, 4, 5, 

<xarray.Dataset> Size: 2GB
Dimensions:               (valid_time: 1, generalVertical: 81, values: 1147980,
                           generalVerticalLayer: 80)
Coordinates:
  * valid_time            (valid_time) datetime64[ns] 8B 2025-01-01T20:00:00
  * generalVertical       (generalVertical) float64 648B 1.0 2.0 ... 80.0 81.0
  * generalVerticalLayer  (generalVerticalLayer) float64 640B 1.0 2.0 ... 80.0
Dimensions without coordinates: values
Data variables:
    W                     (valid_time, generalVertical, values) float64 744MB dask.array<chunksize=(1, 1, 1147980), meta=np.ndarray>
    T                     (valid_time, generalVerticalLayer, values) float64 735MB dask.array<chunksize=(1, 1, 1147980), meta=np.ndarray>
    T_2M                  (valid_time, values) float64 9MB dask.array<chunksize=(1, 1147980), meta=np.ndarray>
    CLCL                  (valid_time, values) float64 9MB dask.array<chunksize=(1, 1147980), meta=np.ndarray>
    TOT_PREC              (valid_time, values) float64 9MB dask.array<chunksize=(1, 1147980), meta=np.ndarray>

In [5]:
ds["T_2M"].values

array([[275.73461914, 275.84399414, 275.67114258, ..., 275.83374023,
        275.7800293 , 275.82543945]], shape=(1, 1147980))

In [6]:
tasks = [dask.delayed(open_virtual_dataset)(url, registry, parser) for url in urls]
vds = dask.compute(*tasks, scheduler="processes")
ds = xr.combine_by_coords(vds, combine_attrs="override")
ds

data=MaskIndex(MaskIndex(GRIBReader(/store_new/mch/msopr/osm/KENDA-CH1/ANA25/det/iaf2025010121),[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1284, 1437, 1473]),[0, 1, 2, 3, 4, 5, 

<xarray.Dataset> Size: 6GB
Dimensions:               (valid_time: 4, generalVertical: 81, values: 1147980,
                           generalVerticalLayer: 80)
Coordinates:
  * valid_time            (valid_time) datetime64[ns] 32B 2025-01-01T20:00:00...
  * generalVertical       (generalVertical) float64 648B 1.0 2.0 ... 80.0 81.0
  * generalVerticalLayer  (generalVerticalLayer) float64 640B 1.0 2.0 ... 80.0
Dimensions without coordinates: values
Data variables:
    W                     (valid_time, generalVertical, values) float64 3GB M...
    T                     (valid_time, generalVerticalLayer, values) float64 3GB ManifestArray<shape=(4, 80, 1147980), dtype=float64, chunks=(1, 1, 11479...
    T_2M                  (valid_time, values) float64 37MB ManifestArray<sha...
    CLCL                  (valid_time, values) float64 37MB ManifestArray<sha...
    TOT_PREC              (valid_time, values) float64 37MB ManifestArray<sha...

In [7]:
# TODO: persist the virtualized dataset using icechunk or kerchunk references
# could not get it to work yet